In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

# Read data
dFrame = pd.read_excel("Data_processed.xlsx")

# Function to clean the data (fill missing values)
def cleanData(dataFrame):
    for column in dataFrame.columns:
        if dataFrame[column].dtype in ['float64', 'int64']:
            # Fill missing numeric values with mean
            dataFrame[column] = dataFrame[column].fillna(dataFrame[column].mean()) 
        else:
            # Fill missing categorical values with mode (most frequent value)
            dataFrame[column] = dataFrame[column].fillna(dataFrame[column].mode()[0])  
    return dataFrame

# Function to convert categorical columns to numeric
def changeToNumeric(dataFrame):
    for col in dataFrame.columns:
        if dataFrame[col].dtype not in ['float64', 'int64']:
            # Convert categorical columns to numeric codes
            dataFrame[col] = dataFrame[col].astype('category').cat.codes
    return dataFrame

# Function to perform feature selection using RFE (Recursive Feature Elimination)
def featureSelection(model, X, y):
    # Create the RFE model and select the top 10 features
    rfe = RFE(model, n_features_to_select=10)
    X_rfe = rfe.fit_transform(X, y)
    selected_features = X.columns[rfe.support_]
    return X_rfe, selected_features

# Function to perform evaluation metrics calculation
def evaluateModel(model, X, y):
    # Stratified 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Metrics lists
    auc_scores = []
    accuracy_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    mcc_scores = []
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc[] for row indexing
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Use .iloc[] for row indexing
        
        # Feature Scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train and Predict
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        # Handle Perceptron which doesn't have predict_proba
        if hasattr(model, 'predict_proba'):
            y_prob = model.predict_proba(X_test_scaled)  # for AUC
        elif hasattr(model, 'decision_function'):
            # Use decision_function for Perceptron as a proxy for probabilities
            y_prob = model.decision_function(X_test_scaled)
            # Convert to probabilities using a sigmoid function (for binary classification)
            if len(np.unique(y)) == 2:
                y_prob = 1 / (1 + np.exp(-y_prob))  # Sigmoid for binary classification
            else:
                # For multi-class classification, we can use the softmax function
                # but let's skip AUC calculation for Perceptron in multi-class case
                y_prob = None  # Set it to None, we will not calculate AUC for multi-class models with decision_function
        
        # Calculate metrics
        if y_prob is not None:  # Only calculate AUC if y_prob is available
            try:
                if len(np.unique(y)) > 2:  # Multi-class classification
                    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')  # Multi-class AUC
                else:  # Binary classification
                    auc_score = roc_auc_score(y_test, y_prob)
                auc_scores.append(auc_score)
            except ValueError:
                # If AUC calculation fails, append NaN (e.g., in case of empty predictions or other issues)
                auc_scores.append(np.nan)
        else:
            auc_scores.append(np.nan)  # If no probability estimate, append NaN
        
        # Calculate other metrics
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
        recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
        mcc_scores.append(matthews_corrcoef(y_test, y_pred))
    
    # Store the results for this model
    results = {
        "Model": model.__class__.__name__,
        "AUC": np.nan if np.isnan(np.mean(auc_scores)) else np.mean(auc_scores),  # Handle NaN AUC
        "Accuracy (CA)": np.mean(accuracy_scores),
        "F1-score": np.mean(f1_scores),
        "Precision": np.mean(precision_scores),
        "Recall": np.mean(recall_scores),
        "MCC": np.mean(mcc_scores)
    }
    
    return results

# List of models to evaluate
models = [
    LogisticRegression(max_iter=10000, solver='liblinear', random_state=42),  # Increased max_iter and solver changed
    SVC(random_state=42, probability=True),  # SVC with probability=True for AUC
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    AdaBoostClassifier(random_state=42, algorithm='SAMME'),
    GaussianNB(),
    KNeighborsClassifier(),
    Perceptron(random_state=42),
]

# Clean the data
cleanedData = cleanData(dFrame)

# Convert non-numeric data to numeric
dFrame = changeToNumeric(cleanedData)

# Split data into features and target
X = dFrame.drop('GrainYield', axis=1)  # Assuming 'GrainYield' is the target column
y = dFrame['GrainYield']

# Initialize a list to store results
all_results = []

# Evaluate each model and store the results
for model in models:
    # Feature selection (RFE) for Logistic Regression (or any other model you choose)
    if model.__class__.__name__ == 'LogisticRegression':
        print(f"Evaluating model {model.__class__.__name__} with feature selection...")
        X_rfe, selected_features = featureSelection(model, X, y)
        print(f"Selected Features: {selected_features}")
        # Evaluate on selected features
        result_with_fs = evaluateModel(model, pd.DataFrame(X_rfe, columns=selected_features), y)
        all_results.append(result_with_fs)
    else:
        print(f"Evaluating model {model.__class__.__name__} without feature selection...")
        result = evaluateModel(model, X, y)
        all_results.append(result)

# Create a DataFrame to display results as a table
results_df = pd.DataFrame(all_results)

# Display the results
print("Model Evaluation Results:")
print(results_df)

# Sort by any column, by AUC
results_df = results_df.sort_values(by="AUC", ascending=False)
print("\nSorted by AUC:")
print(results_df)

Model Evaluation Results:
                        Model       AUC  Accuracy (CA)  F1-score  Precision  \
0          LogisticRegression  0.835153       0.687015  0.683414   0.686735   
1                         SVC  0.883250       0.749933  0.744568   0.753308   
2      DecisionTreeClassifier  0.757101       0.689424  0.689432   0.689639   
3      RandomForestClassifier  0.908090       0.771084  0.769778   0.771344   
4  GradientBoostingClassifier  0.886784       0.751807  0.748617   0.752890   
5          AdaBoostClassifier  0.785572       0.628916  0.615330   0.633552   
6                  GaussianNB  0.663523       0.310040  0.232742   0.570041   
7        KNeighborsClassifier  0.872707       0.732262  0.731896   0.734453   
8                  Perceptron       NaN       0.598929  0.594406   0.595829   

     Recall       MCC  
0  0.687015  0.490297  
1  0.749933  0.593792  
2  0.689424  0.503298  
3  0.771084  0.630562  
4  0.751807  0.597788  
5  0.628916  0.384913  
6  0.310040  0.